In [78]:
from textgenrnn import textgenrnn
word_level_model = textgenrnn('nelsonweights/intro_model_weights.hdf5',
                              config_path='nelsonweights/intro_model_config.json',
                              vocab_path='nelsonweights/intro_model_vocab.json')

Using TensorFlow backend.


In [79]:
lines = word_level_model.generate(50, temperature=1.0, max_gen_length=100, return_as_list=True)




  0%|                                                                                           | 0/50 [00:00<?, ?it/s]


  2%|█▋                                                                                 | 1/50 [00:09<07:45,  9.49s/it]


  4%|███▎                                                                               | 2/50 [00:12<06:01,  7.54s/it]


  6%|████▉                                                                              | 3/50 [00:17<05:15,  6.71s/it]


  8%|██████▋                                                                            | 4/50 [00:22<04:51,  6.34s/it]


 10%|████████▎                                                                          | 5/50 [00:32<05:35,  7.46s/it]


 12%|█████████▉                                                                         | 6/50 [00:43<06:07,  8.36s/it]


 14%|███████████▌                                                                       | 7/50 [00:46<04:56,  6.89s/it]


 16%|█████████████▎  

In [72]:
print (lines)

['\n', '\nthree, on my am t whole on, kissing from am from\nlights from\nam\nt\nfor running the, five for\nrunning five up the for in\n\ntime the\n\n\n\nfrom\n\n, just the no almost']


In [62]:
word_level_model.generate_samples(temperatures=[0.5, 0.6, 0.7, 0.8, 0.9])

####################
Temperature: 0.5
####################

i m remember you how were
when you night you the you
forget you you you the you
you we
,
you you, you your we when when when were
night when when you you you you to to


you little little young, my taxes the the finished
a an finished finished finished finished the the in the finished the the the finished beat by the an
the by finished finished tortoise a a a a
a

she a a she
no no, is she a us she us is there there she she she
she a, grind -
grind strong
pressure you and
we of out enough love you to make rear you you
to you
it

the your


enough we

we we bare we
the your (the have have (eyes (((into (bringing - the of - up starting world shit - starting - home of its a - starting brand
with
with


shit
a
up
a





a

the






start
a
start ice every every hour away that

it
that



that
every



every



if day
,

that
away
missing the every


day
text

,


you

you








,
missing
you



i m t know you the
,

i


you a y


hit the floor pull close you show you
you you you all you let you
you
even, you
you you
you
you




you
you you you up

to
you we
and lovers
at lie love


you wanna to you so if make i em em booty style check,,
it pop,, this,,
show



In [80]:
import rhymes

# print (lines)
lines2 = list()

for line in lines:
    split = line.splitlines()
    for l in split:
        lines2.append(l)
print(lines2)

rhyme_dict = dict()

#Generate of list of last words from each line
for index, line in enumerate(lines2):
    if (len(line) == 0):
        continue
    try:
        wordsInLine = line.split()
        curr_word = ""
        wordsInLine[-1] = wordsInLine[-1].lower()
        #remove puncuation
        for c in wordsInLine[-1]:
            if ord(c) >= 97 and ord(c) <= 122:
                curr_word += c
                
        #only add words (remove any empty strings)
        if curr_word != "":
            IT_RHYMES_BABYYY = False
            for key in rhyme_dict.keys():
                if (rhymes.doTheyRhyme(curr_word, key)):
                    rhyme_dict[key].append(index)
                    IT_RHYMES_BABYYY = True
            if (not IT_RHYMES_BABYYY):
                rhyme_dict[curr_word] = [index]
    except:
        print (curr_word, ' was not found in the dictionary')
        continue
print (rhyme_dict)

['', 'if wish to i, wish that', '', 'that boy with could could i see i', 'could existed stay let was', 'nobody nothing want was', ', i everyday wants, now', 'remember', 'how the out this', '', 'nobody like out', 'like help your', 'this', 'everyday it to s in but step rocking a that', 'that', ', e into -', 'off, em talkin right so style and got i the 2 the of was', 'me car say,', ', yourself when can well', '', 'if you me you you strange s but but, one one in to more thang like never that that i', 'but, thang but', '', 'maybe when all feeling hard it s not not not this s and s what hard not, there but just not i not make', ': you and i not not i you but it not you rappers like what searching in', '', 'it s hot young should t ride ya it gimme, a from water light the kind run stop the crib s do the the the the a the the the -', 'the from the the water a', ', will the the the do about make', 'seen', 'you distant used', '', 'how to the', '', 'the', 'and', 'to', '', '', '', '', 'how in want 

niggas  was not found in the dictionary
dont  was not found in the dictionary
{'that': [1, 13, 14, 116, 158, 167, 184, 197, 198, 245, 280, 285], 'i': [3, 20, 57, 85, 115, 123, 146, 149, 156, 164, 168, 188, 190, 191, 223, 227, 238, 254, 263, 313, 350, 359, 370, 406, 410, 414, 420, 422, 439], 'was': [4, 5, 16, 211, 224, 451], 'now': [6, 41, 43, 435], 'remember': [7], 'this': [8, 12, 59, 81, 84, 97, 127, 145, 231, 299, 369, 411, 415], 'out': [10, 86, 212, 251], 'your': [11, 76, 89, 92, 128, 150, 196, 304, 334, 392, 399], 'say': [17, 199, 384], 'well': [18, 83, 187, 201, 221, 351, 448], 'but': [21, 64, 88, 226, 289, 311, 327, 345, 346, 361, 373], 'make': [23, 28, 79, 119, 193, 354], 'in': [24, 54, 129, 206], 'a': [27, 32, 34, 44, 48, 50, 52, 56, 62, 75, 122, 126, 133, 138, 180, 182, 220, 260, 265, 287, 323, 328, 385, 388, 409, 438], 'seen': [29, 91], 'used': [30], 'and': [35, 117, 160, 319, 419, 421], 'to': [36, 45, 74, 90, 93, 102, 104, 142, 154, 157, 161, 174, 175, 176, 186, 208, 218, 24

In [81]:
# remove lines that don't rhyme with any other line

import importlib
importlib.reload(rhymes)

ordered_rhymes = []
to_remove = []
for key in rhyme_dict.keys():
    if len(rhyme_dict[key]) < 2:
        to_remove.append(key)
        continue
    for index in rhyme_dict[key]:
        ordered_rhymes.append(lines2[index])
        
for word in to_remove:
    rhyme_dict.pop(word)
    
rhyme_count = rhymes.howManyRhymes(ordered_rhymes)

In [8]:
# given rhyming scheme input, output generated lines with given rhyming scheme

import re

print ('What rhyming scheme would you like to use? (A to Z)')
rhyme_scheme = input()
if (not re.fullmatch('[A-Z]+', rhyme_scheme)):
    print ('That is not a valid rhyming scheme.')
    exit()
    
# count the required number of lines for each letter/section of the rhyming scheme
rhyme_counts = dict()
for char in rhyme_scheme:
    if char in rhyme_counts:
        rhyme_counts[char] += 1
    else:
        rhyme_counts[char] = 1
print (rhyme_counts)

# map each letter/section of the rhyming scheme to a rhyme in rhyme_dict
used_rhymes = set()
letter_to_rhyme = dict()
for letter in rhyme_counts.keys():
    for rhyme in rhyme_dict.keys():
        # if the current rhyme has enough rhymes for the current letter in the rhyming scheme
        # AND the rhyme hasn't been used, use it
        if (len(rhyme_dict[rhyme]) >= rhyme_counts[letter] and rhyme not in used_rhymes):
            letter_to_rhyme[letter] = rhyme_dict[rhyme]
            used_rhymes.add(rhyme)
            break
            
print (letter_to_rhyme)

# print the results!
rhyme_indices = dict()
for char in rhyme_scheme:
    if (char not in rhyme_indices):
        print (lines2[letter_to_rhyme[char][0]])
        rhyme_indices[char] = 1
    else:
        print (lines2[letter_to_rhyme[char][rhyme_indices[char]]])
        rhyme_indices[char] += 1

What rhyming scheme would you like to use? (A to Z)


In [ ]:
# intro AABAAB
# chorus AABAAB
# verse AAAAAA
# bridge CCCADD